# Develop, evaluate, and score a forecasting model for electricity generation

In [ ]:
"""
 Introduction

In this notebook, you'll see Microsoft Fabric's end-to-end data science workflow for a forecasting model. This scenario uses the historic sales data to predict the sales for different categories of products at a superstore.

Forecasting is a crucial asset in sales, harnessing historical data and predictive methods to provide insights into future trends. By analyzing past sales, identifying patterns, and learning from consumer behavior, businesses can optimize inventory, production, and marketing strategies. This proactive approach enhances adaptability, responsiveness, and overall performance of businesses in a dynamic marketplace.

The main steps in this notebook are:

1. Load the data
2. Understand and process the data using exploratory data analysis
3. Train a machine learning model using an open source software package called `SARIMAX` and track experiments using MLflow and Fabric Autologging feature
4. Save the final machine learning model and make predictions
5. Demonstrate the model performance via visualizations in Power BI

Thesis

What percentage of fuel generation will come from Natural Gas in the coming months?

"""

In [ ]:
# Record the start time
start_time = time.time()

In [ ]:
pip install pmdarima


In [ ]:
pip install bayesian-optimization

## Libraries and Variables

In [ ]:
# Importing required libraries
from datetime import timedelta
# Import required libraries for model evaluation
from pmdarima.arima import auto_arima
import pmdarima as pm
from pprint import pprint
import random
import os, requests
import warnings
import itertools
from itertools import product
# Record the notebook running time
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.x13 import x13_arima_select_order
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
# Import required libraries for model evaluation
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, ParameterGrid
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import regexp_extract
from bayes_opt import BayesianOptimization

In [ ]:
# Set up MLflow for experiment tracking
import mlflow
from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.models import Model

"""
Define Variables
"""

EXPERIMENT_NAME = "EIA-forecasting-bayesian"  # MLflow experiment name

"""
ML Flow: Machine Learning

Autologging in Microsoft Fabric extends the MLflow autologging capabilities by automatically capturing the values of input parameters and output metrics of a machine learning model as it is being trained. This information is then logged to the workspace, where it can be accessed and visualized using the MLflow APIs or the corresponding experiment in the workspace. To learn more about autologging, see [Autologging in Microsoft Fabric](https://aka.ms/fabric-autologging).
"""

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.autolog(disable=True)  # Disable MLflow autologging



## Step 1: Datasets

### Data Retrieval

In [ ]:
"""
Step 1: Read the dataset from the lakehouse
"""

df = spark.sql(
"""
SELECT 
Type.FuelClass Fuel_class
,Ops.date_full Operating_date
,sum(Fact.NetGenerationElectricityMWh) Electricity_generated
FROM LH_EIA_GOLD.fact_generation Fact
left outer join dim_fuel_type Type on Fact.FuelTypeKey = Type.FuelTypeKey
left outer join dim_date Ops on Fact.SurveyDateKey = Ops.date_id 
where Ops.date_full > '1999-12-31'
GROUP BY 
Type.FuelClass 
,Ops.date_full 
"""
)

Raw_Data = df


"""
Data Reformatting
"""
Raw_Data = Raw_Data.withColumnRenamed("Fuel_Class", "Fuel class")
Raw_Data = Raw_Data.withColumnRenamed("Operating_Date", "Operating date")
Raw_Data = Raw_Data.withColumnRenamed("Electricity_Generated", "Electricity generated")
Raw_Data = Raw_Data.withColumn("Operating date", to_date(col("Operating date")))
# Convert the PySpark DataFrame to a Pandas DataFrame
Raw_Data = Raw_Data.toPandas()

"""
How to specify or convert the data types post-conversion in the Pandas DataFrame
"""

# Assuming Training_data is your Pandas DataFrame
Raw_Data['Fuel class'] = Raw_Data['Fuel class'].astype(str)
# Assuming Training_data is your Pandas DataFrame
Raw_Data['Operating date'] = pd.to_datetime(Raw_Data['Operating date'])
# Convert the 'Electricity generated' column to int
Raw_Data['Electricity generated'] = Raw_Data['Electricity generated'].astype(float)

display(Raw_Data)

In [ ]:
# Data preprocessing
PreProcessed_Data = Raw_Data.sort_values('Operating date')
PreProcessed_Data.isnull().sum()
PreProcessed_Data = PreProcessed_Data.groupby('Operating date')['Electricity generated'].sum().reset_index()
PreProcessed_Data = PreProcessed_Data.set_index('Operating date')

display(PreProcessed_Data)
print(PreProcessed_Data)

In [ ]:
# Data Processing
Overall_Data_processed = PreProcessed_Data.fillna(0) # Fill NaN values with zero
Overall_Data_processed = Overall_Data_processed.astype('int64')
Overall_Data_processed = Overall_Data_processed.reset_index()
Overall_Data_processed['Operating date'] = pd.to_datetime(Overall_Data_processed['Operating date'])
display(Overall_Data_processed)
print(Overall_Data_processed)

In [ ]:
# Interpolated_data

"""
Interpolated data refers to values that have been estimated or calculated within a range based on known data points using techniques like linear interpolation, polynomial interpolation, spline interpolation, etc.
"""

### Training Data

In [ ]:
# 2022 Data
start_date = "2010-01-01"
end_date = "2022-01-01"

Training_data = Overall_Data_processed[
    (Overall_Data_processed['Operating date'].dt.strftime('%Y-%m-%d') >= start_date) &
    (Overall_Data_processed['Operating date'].dt.strftime('%Y-%m-%d') < end_date)
]

display(Training_data)
Training_data = Training_data.set_index(['Operating date'])
display(Training_data)

### Validation Data

In [ ]:
# 2022 Data
start_date = "2022-01-01"
end_date = "2023-01-01"

validation_data = Overall_Data_processed[
    (Overall_Data_processed['Operating date'].dt.strftime('%Y-%m-%d') >= start_date) &
    (Overall_Data_processed['Operating date'].dt.strftime('%Y-%m-%d') < end_date)
]

display(validation_data)
validation_data = validation_data.set_index(['Operating date'])
display(validation_data)

### Test Data

In [ ]:
test_data = Overall_Data_processed

display(test_data)
test_data = test_data.set_index(['Operating date'])
display(test_data)

## Step 2: Statistical Analysis

In [ ]:
"""
Statistical analysis

A time series tracks four data elements at set intervals in order to determine the variation of those four elements in the time series pattern. These elements include:

- **Level:** Refers to the fundamental component that represents the average value for a specific time period.

- **Trend:** Describes whether the time series is decreasing, constant, or increasing over time.

- **Seasonality:** Describes the periodic signal in the time series and looks for cyclic occurrences that affect the time series' increasing or decreasing patterns.

- **Noise/Residual:** Refers to the random fluctuations and variability in the time series data that cannot be explained by the model.

In the following, you will observe the above four components for your dataset after the pre-processing.
"""


# Decompose the time series into its components using statsmodels
result = sm.tsa.seasonal_decompose(Training_data, model='additive')

# Labels and corresponding data for plotting
components = [('Seasonality', result.seasonal),
              ('Trend', result.trend),
              ('Residual', result.resid),
              ('Observed Data', Training_data)]

# Create subplots in a grid
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 7))
plt.subplots_adjust(hspace=0.8)  # Adjust vertical space
axes = axes.ravel()

# Plot the components
for ax, (label, data) in zip(axes, components):
    ax.plot(data, label=label, color='blue' if label != 'Observed Data' else 'purple')
    ax.set_xlabel('Time')
    ax.set_ylabel(label)
    ax.set_xlabel('Time', fontsize=10)
    ax.set_ylabel(label, fontsize=10)
    ax.legend(fontsize=10)

plt.show()

"""
Doing time series analysis. We are preforming seasonal decomposition. This gives an overall understanding of how these components contribute to the entire time series.
Understanding the seasonality, trend, and noise in the forecasting data through the above plots allows to capture underlying patterns, and develop models that make more accurate predictions that are resilient to random fluctuations.
"""

In [ ]:
"""
Statistical Measure of accuracy
"""
def mean_absolute_error(y_true, y_pred):
    """
    Calculate Mean Absolute Error (MAE)
    
    Args:
    y_true: List or array containing true values
    y_pred: List or array containing predicted values
    
    Returns:
    mae: Mean Absolute Error
    """
    if len(y_true) != len(y_pred):
        raise ValueError("Lengths of y_true and y_pred must be the same.")
    
    total_error = 0
    for true_val, pred_val in zip(y_true, y_pred):
        total_error += abs(true_val - pred_val)
    
    mae = total_error / len(y_true)
    return mae

## Step 3: Model Training and Tracking

With your data in place, you can define the forecasting model. Apply the Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors (SARIMAX) in this notebook. SARIMAX is a time series forecasting model that extends SARIMA to include exogenous variables. It combines autoregressive (AR) and moving average (MA) components, seasonal differencing, and external predictors to make accurate and flexible forecasts for time series data, making it a powerful tool for various forecasting tasks.

You will also use MLfLow and Fabric Autologging to track the experiments. Here you'll load the delta table from the lakehouse. You may use other delta tables considering the lakehouse as the source.

"""
SARIMAX, or Seasonal Autoregressive Integrated Moving Average with Exogenous Factors, 
a time series forecasting method that extends the traditional ARIMA model to include seasonal components and exogenous variables. 

The acronym breaks down as follows:
- Seasonal: Accounts for patterns that repeat over known intervals or seasons.
- ARIMA: Stands for Autoregressive Integrated Moving Average, a model used for understanding and forecasting time series data.
- X: Denotes the inclusion of exogenous variables, which are external factors that can influence the time series being forecasted.

In SARIMAX models, the seasonal and non-seasonal components of the time series data are modeled separately. 
It's a powerful tool for handling time series data that exhibits both seasonal patterns and relationships with other variables.
"""

### Hyperparameters

#### Definition:

- Hyperparameters are settings or configurations that are set before the model is trained and are not directly learned from the data during the training process. 
- They determine the structure or behavior of the model and need to be chosen or tuned based on domain knowledge, heuristics, or optimization techniques. 
- In the context of SARIMAX, these parameters define the characteristics of the seasonal and non-seasonal components of the time series model.

### Hyperparameter tuning

SARIMAX takes into account the parameters involved in regular ARIMA mode `(p,d,q)` and also adds the seasonality parameters `(P,D,Q,s)`. These arguments to SARIMAX model are called order `(p,d,q)` and seasonal order `(P,D,Q,s)` respectively and hence 7 parameters to tune. Prior to model training, you need to set up these parameters which are defined in the following.

#### Order Parameters `(p, d, q)`:
- `p`: The order of the autoregressive (AR) component, indicating how many past observations are considered. It is also known as the AR order.
- `d`: The degree of differencing required to make the time series stationary. It is also known as the differencing order.
- `q`: The order of the moving average (MA) component, indicating how many past white noise error terms are considered. It is also known as the MA order.

#### Seasonal Order Parameters `(P, D, Q, s)`:

- `P`: The seasonal order of the autoregressive (AR) component, similar to `p` but for the seasonal part.
- `D`: The seasonal order of differencing, similar to `d` but for the seasonal part.
- `Q`: The seasonal order of the moving average (MA) component, similar to `q` but for the seasonal part.
- `s`: The number of time steps per seasonal cycle (e.g., 12 for monthly data with a yearly seasonality).


 The autoregressive order `p` represents the number of past observations in the time series that are used to predict the current value. Typically, `p` should be a non-negative integer. Common values for `p` are usually in the range of 0 to 3, although higher values are possible depending on the specific characteristics of the data. A higher p indicates a longer memory of past values in the model.

 The moving average order `q` represents the number of past white noise error terms that are used to predict the current value. Similar to `p`, `q` should also be a non-negative integer. Common values for `q` are typically in the range of 0 to 3, but higher values may be necessary for certain time series. A higher `q` indicates a stronger reliance on past error terms to make predictions.

 The differencing order `d` represents the number of times the time series needs to be differenced to achieve stationarity. `d` should be a non-negative integer. Common values for `d` are usually in the range of 0 to 2. A `d` value of 0 means the time series is already stationary, while higher values indicate the number of differencing operations required to make it stationary.


#### Determine Optional Parameters:
- The `enforce_stationarity` parameter controls whether or not the model should enforce stationarity on the time series data before fitting the SARIMAX model. When `enforce_stationarity` is set to `True` (the default), it indicates that the SARIMAX model should enforce stationarity on the time series data. This means that the SARIMAX model will automatically apply differencing to the data to make it stationary, as specified by the `d` and `D` orders, before fitting the model. This is a common practice because many time series models, including SARIMAX, assume that the data is stationary. If your time series is non-stationary (e.g., it exhibits trends or seasonality), it is generally a good practice to set `enforce_stationarity` to `True` and let the SARIMAX model handle the differencing to achieve stationarity. If your time series is already stationary (e.g., it has no trends or seasonality), you can set `enforce_stationarity` to `False` to avoid unnecessary differencing.

- The `enforce_invertibility` parameter controls whether or not the model should enforce invertibility on the estimated parameters during the optimization process. When `enforce_invertibility` is set to `True` (the default), it indicates that the SARIMAX model should enforce invertibility on the estimated parameters. Invertibility ensures that the model is well-defined and that the estimated autoregressive (AR) and moving average (MA) coefficients are within the range of stationarity. Enforcing invertibility is typically recommended to ensure that the SARIMAX model adheres to the theoretical requirements for a stable time series model and helps prevent issues with model estimation and stability.

- The default is an `AR(1)` model which refers to `(1,0,0)`. However, keep in mind that the appropriate values for `p`, `q`, and `d` can vary from one time series to another, and determining the optimal values often involves analyzing the autocorrelation and partial autocorrelation functions (ACF and PACF) of the time series data and using model selection criteria like AIC or BIC. It's common practice to try different combinations of p, q, and d and evaluate the model's performance for a given dataset. Note that the parameters for the seasonal order `(P, D, Q, s)` is similar in concept to the non-seasonal order parameters `(p, q, d)`, hence it is avoided explaining in detail again. 

In [ ]:
def check_stationarity(data):
    result = adfuller(data)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

# Call the function with your time series data 'y'
check_stationarity(Training_data)


#### specifying the parameter grid:
- For SARIMAX or any other model with hyperparameters, setting up a parameter grid involves specifying various potential values or ranges for the parameters you aim to optimize. For instance, in SARIMAX, you might define a grid or search space for the order parameters (p, d, q) and seasonal order parameters (P, D, Q, s). 

- This grid represents various combinations of these hyperparameters that you'll test during the hyperparameter search process. The model will be trained and evaluated for each combination in this grid to determine the combination that performs best according to the chosen evaluation metric.
 
- The objective is to search through this space efficiently to find the set of hyperparameters that optimizes the model's performance without overfitting to the training data. Techniques like grid search, random search, or more advanced optimization algorithms are used to navigate this parameter space and identify the best hyperparameter values for the model.


In [ ]:
# First way to define the parameter grid
param_grid = {
    'order': [(p, d, q) for p, d, q in product(range(0, 8), repeat=3)],
    'seasonal_order': [(x[0], x[1], x[2], 12) for x in product(range(0, 8), repeat=3)]
}

# Use ParameterGrid to generate combinations
grid = ParameterGrid(param_grid)

# Get the total number of combinations
total_combinations = len(grid)

print(f"Total number of combinations: {total_combinations}")

print(type(param_grid)) 

print(type(grid))  



In [ ]:
# Another way to define the parameter grid

p_values = list(range(8))  # Range from 0 to 7
d_values = list(range(4))  # For instance, range from 0 to 3
q_values = list(range(8))  # For instance, range from 0 to 7
P_values = list(range(8))  # For instance, range from 0 to 7
D_values = list(range(3))  # For instance, range from 0 to 2
Q_values = list(range(8))  # For instance, range from 0 to 7
s_values = list ([12])  # Value should just be 12.

# Create the parameter grid
parameter_grid = list(product(p_values, d_values, q_values, P_values, D_values, Q_values, s_values))

total_combinations = len(parameter_grid)

print(f"Total number of combinations: {total_combinations}")


#### Proof of Concept: Bayesian Optimization

In [ ]:
# Define your function to optimize (replace with your actual function)
def evaluate_parameters(p, d, q, P, D, Q, s):
    try:
        # Here you'll train your SARIMAX model with the given parameters
        model = SARIMAX(Training_data, order=(int(p), int(d), int(q)), seasonal_order=(int(P), int(D), int(Q), int(s)))
        fitted_model = model.fit(disp=False)
        
        # # Make predictions
        
        forecast = fitted_model.get_forecast(steps=len(validation_data))
        forecast_values = forecast.predicted_mean

        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(validation_data, forecast_values))

        # Return negative RMSE (maximize)
        neg_RMSE = -1.0 * rmse
        return neg_RMSE if 0.2 <= rmse <= 0.5 else float(rmse)

        """
        In Bayesian Optimization, the objective is typically to maximize a function. 
        By returning the negative RMSE, the optimizer effectively maximizes the negative RMSE
        This is equivalent to minimizing the positive RMSE.
        """
    except Exception as e:
        # Handle potential exceptions and return negative infinity
        print(f"An error occurred: {str(e)}")
        return float('-inf')


# Define the bounds for the hyperparameters
pbounds = {
    'p': (0, 7),
    'd': (0, 3),
    'q': (0, 7),
    'P': (0, 7),
    'D': (0, 2),
    'Q': (0, 7),
    's': (12, 12)  # Single value of 12
}

# Ensure only integer values for hyperparameters
for key in pbounds:
    pbounds[key] = (int(pbounds[key][0]), int(pbounds[key][1]))
    
# Calculate the total number of combinations
total_combinations = 1
for param_values in pbounds.values():
    total_combinations *= len(range(param_values[0], param_values[1] + 1))


print(f"Total number of combinations: {total_combinations}")


# Initialize BayesianOptimization
optimizer = BayesianOptimization(f=evaluate_parameters, pbounds=pbounds, random_state=42)

# # Perform optimization
optimizer.maximize(init_points=5, n_iter=20)  # Adjust init_points and n_iter as needed

# n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.

"""This parameter specifies the total number of iterations (steps) of Bayesian Optimization. 
Each iteration involves proposing a new set of hyperparameters to evaluate based on previous evaluations. 
Increasing `n_iter` allows the optimizer to explore more points in the search space and potentially find better-performing hyperparameter sets. 
However, a higher `n_iter` also means more computational time."""

# init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
# This parameter determines the number of initial random points to explore in the search space before the Bayesian Optimization algorithm starts to exploit and explore based on the observed results.

"""
The total number of iterations will be the sum of init_points and n_iter.
"""

In [ ]:
# list of all parameters probed and their corresponding target value
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

In [ ]:
# The best combination of parameters and target value found
print(optimizer.max)

In [ ]:
# The best combination of parameters and target value found
data = (optimizer.max)

# Unpack the nested dictionary
params = data['params']

# Create individual variables
p = int(params['p'])
d = int(params['d'])
q = int(params['q'])
P = int(params['P'])
D = int(params['D'])
Q = int(params['Q'])
s = int(params['s'])

# Print the variables
print(f"p = {p}")
print(f"d = {d}")
print(f"q = {q}")
print(f"P = {P}")
print(f"D = {D}")
print(f"Q = {Q}")
print(f"s = {s}")


#### Bayesian Optimization version 2

In [ ]:
# # Testing RMSE and a larger space.

# # Define the bounds for the hyperparameters
# pbounds = {
#     'p': (0, 7),
#     'd': (0, 3),
#     'q': (0, 7),
#     'P': (0, 7),
#     'D': (0, 2),
#     'Q': (0, 7),
#     's': (12, 12)  # Single value of 12
# }

# # Calculate the total number of combinations
# total_combinations = 1
# for param_values in pbounds.values():
#     total_combinations *= len(range(param_values[0], param_values[1] + 1))

# print(f"Total number of combinations: {total_combinations}")

# total_combinations = 50000

# num_iterations = .10 * total_combinations

# init_points= .10 * num_iterations

# n_iter= num_iterations - init_points

# init_points = int(init_points)
# n_iter = int(n_iter)

# # Define your function to optimize (replace with your actual function)
# def evaluate_parameters(p, d, q, P, D, Q, s):
#     try:
#         # Here you'll train your SARIMAX model with the given parameters
#         model = SARIMAX(Training_data, order=(int(p), int(d), int(q)), seasonal_order=(int(P), int(D), int(Q), int(s)))
#         fitted_model = model.fit(disp=False)
        
#         # Make predictions
#         forecast = fitted_model.get_forecast(steps=len(validation_data))
#         forecast_values = forecast.predicted_mean

#         # Calculate RMSE
#         rmse = np.sqrt(mean_squared_error(validation_data, forecast_values))

#         # Return negative RMSE (maximize)
#         neg_RMSE = -1.0 * rmse

#         # Stop the optimization if the condition is met
#         if 0.2 <= rmse <= 0.5:
#             raise StopIteration(f"Optimization stopped. Desired RMSE condition met: {rmse}")

#         return rmse

#     except Exception as e:
#         # Handle potential exceptions and return negative infinity
#         print(f"An error occurred: {str(e)}")
#         return float('-inf')

# # Initialize BayesianOptimization
# optimizer = BayesianOptimization(f=evaluate_parameters, pbounds=pbounds, random_state=42)

# try:
#     # Perform optimization
#     results = optimizer.maximize(init_points=init_points, n_iter=n_iter)
# except StopIteration as e:
#     print(e)



## Step 4: Score the model and save predictions

Scoring a model typically refers to evaluating its performance or assessing how well it predicts or fits the data it was trained on or applied to. For instance, in the context of a SARIMAX (Seasonal Autoregressive Integrated Moving Average with Exogenous Factors) model, scoring could involve several evaluation metrics.

Here are a few common ways to score a SARIMAX model:

1. **Mean Squared Error (MSE):** Measures the average of the squared differences between predicted and actual values. Lower MSE indicates better performance.

2. **Root Mean Squared Error (RMSE):** The square root of the MSE, giving an error value in the same units as the target variable. Lower RMSE signifies better performance.

3. **Mean Absolute Error (MAE):** Measures the average of the absolute differences between predicted and actual values. Similar to MSE but less sensitive to outliers.

4. **AIC (Akaike Information Criterion) or BIC (Bayesian Information Criterion):** These are used for model selection among a set of models. Lower AIC or BIC values indicate a better fit, with a balance between goodness of fit and model complexity.

5. **R-squared (R²) or adjusted R-squared:** Measures the proportion of variance in the dependent variable that is predictable from the independent variables. Higher R² values indicate a better fit of the model to the data.

When applying a SARIMAX model, you'd use historical data for training, leaving out a portion (the validation or test set) to evaluate its predictive performance. After fitting the model on the training data, you'd use the test set to generate predictions and then compare these predictions to the actual values to calculate these scoring metrics.

These scores help to gauge how well the model is performing and whether it needs adjustments or fine-tuning to improve its predictions.

In [ ]:
validation_data['Electricity generated'] = pd.to_numeric(validation_data['Electricity generated'], errors='coerce')

# Start an MLflow run
with mlflow.start_run():

    mod = sm.tsa.statespace.SARIMAX(test_data,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, s),
                                enforce_stationarity=True,
                                enforce_invertibility=True)
    results = mod.fit(disp=False)
    # Forecasting on the Historical dataset
    forecast = results.get_forecast(steps=len(validation_data))
    # Forecasting on the validation dataset (2022)
    validation_amt = results.get_forecast(steps=len(validation_data))

    # Compare forecast to actuals for evaluation
    forecast_values = forecast.predicted_mean
    validation_values = validation_amt.predicted_mean


    # Log SARIMAX model parameters
    mlflow.log_params({"order": (p, d, q), "seasonal_order": (P, D, Q, s), 'enforce_stationarity': True, 'enforce_invertibility': True})
    mape = mean_absolute_percentage_error(validation_data['Electricity generated'], validation_values) * 100
    mae = mean_absolute_error(validation_data['Electricity generated'], validation_values)
    rmse = np.sqrt(mean_squared_error(validation_data, forecast_values))
    mlflow.log_metric("MAPE", mape)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("RMSE", rmse)




# Log the model and parameters
model_name = f"{EXPERIMENT_NAME}-Sarimax"
with mlflow.start_run(run_name="Sarimax") as run:
    mlflow.statsmodels.log_model(results,model_name,registered_model_name=model_name)
    mlflow.log_params({"order":(p,d,q),"seasonal_order":(P, D, Q, s),'enforce_stationarity':True,'enforce_invertibility':True})
    mlflow.log_metric("MAPE", mape)
    mlflow.log_metric("MAE", mae)  
    mlflow.log_metric("RMSE", rmse)              
    model_uri = f"runs:/{run.info.run_id}/{model_name}"
    print("Best model saved in run %s" % run.info.run_id)
    print(f"Best model URI: {model_uri}")
    mlflow.end_run()

# Load the saved model
loaded_model = mlflow.statsmodels.load_model(model_uri)

# Validation Steps
Historical = pd.DataFrame(test_data).reset_index()
Historical.sort_values('Operating date')
display(Historical)

Future = pd.DataFrame(forecast_values).reset_index()
# Get the maximum date from Historical DataFrame
max_date = Historical['Operating date'].max()
num_of_months = 12  # Number of months you want to increment
dates = pd.date_range(start=max_date + pd.offsets.MonthBegin(), periods=num_of_months, freq='MS')
# Assign the generated dates to the 'Operating date' column in the Future DataFrame
Future['Operating date'] = dates
Future = Future.drop('index', axis=1)
Future = Future.reindex(columns=['Operating date','predicted_mean'])
Future.rename(columns={'predicted_mean': 'Electricity generated'}, inplace=True)
display(Future)

result = pd.concat([Historical,Future])
display(result)
result['Order'] = result.apply(lambda row: f"SARIMAX({p}, {d}, {q})x({P}, {D}, {Q}, {s})", axis=1)
# Calculate the Mean Absolute Percentage Error (MAPE) between the 'Actual_Sales' and 'Forecasted_Sales' 
result['MAPE'] = np.NAN
result['MAPE'] = mape
result['MAE'] = mae
result = result.reindex(columns=['Order','Operating date', 'Electricity generated', 'MAPE', 'MAE'])

result.rename(columns={'Operating date': 'Operating_Date'}, inplace=True)
result.rename(columns={'Electricity generated': 'Electricity_generated'}, inplace=True)
result.rename(columns={'predicted_mean': 'Forecast_Electricity'}, inplace=True)
display(result)

# Write Back the results into the lakehouse
table_name = "Demand_Forecast_New_1"
spark.createDataFrame(result).write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")

### Testing

In [ ]:
# Print the variables
print(f"p = {p}")
print(f"d = {d}")
print(f"q = {q}")
print(f"P = {P}")
print(f"D = {D}")
print(f"Q = {Q}")
print(f"s = {s}")


# Start an MLflow run
with mlflow.start_run():

    mod = sm.tsa.statespace.SARIMAX(Training_data,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, s),
                                enforce_stationarity=True,
                                enforce_invertibility=True)
    results = mod.fit(disp=False)
    # Forecasting on the Historical dataset
    forecast = results.get_forecast(steps=len(validation_data))
    # Forecasting on the validation dataset (2022)
    validation_amt = results.get_forecast(steps=len(validation_data))

    # Compare forecast to actuals for evaluation
    forecast_values = forecast.predicted_mean
    validation_values = validation_amt.predicted_mean

# Validation Steps
Historical = pd.DataFrame(Training_data).reset_index()
Historical.sort_values('Operating date')

Future = pd.DataFrame(forecast_values).reset_index()
# Get the maximum date from Historical DataFrame
max_date = Historical['Operating date'].max()
num_of_months = 12  # Number of months you want to increment
dates = pd.date_range(start=max_date + pd.offsets.MonthBegin(), periods=num_of_months, freq='MS')
# Assign the generated dates to the 'Operating date' column in the Future DataFrame
Future['Operating date'] = dates
Future = Future.drop('index', axis=1)
Future = Future.reindex(columns=['Operating date','predicted_mean'])
Future.rename(columns={'predicted_mean': 'Predicted Electricity generated'}, inplace=True)

validation_data = pd.DataFrame(validation_data).reset_index()
# Perform a left join on the 'ID' column
result_df = pd.merge(validation_data, Future, on='Operating date', how='left')

# Rename columns
result_df = result_df.rename(columns={'Electricity generated': 'Actuals', 'Predicted Electricity generated': 'Forecast'})

# Convert 'Actual' and 'Forecast' columns to integers
result_df['Actuals'] = result_df['Actuals'].astype(int)
result_df['Forecast'] = result_df['Forecast'].astype(int)

# Calculate the new column as a percentage
result_df['ActualOverForecastMinus1'] = (result_df['Actuals'] / result_df['Forecast'] - 1) * 100

# Display the DataFrame with the updated columns
print(result_df)



In [ ]:
# Record the end time
end_time = time.time()

# Calculate the total execution time
total_time = end_time - start_time

# Display the DataFrame and total execution time
print(f"\nTotal execution time: {total_time} seconds")